In [ ]:
#Ali Badrloo
#Mahdi Alikhani
#Mohammadjavad Ghazikhani

In [2]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.image import resize
from tensorflow.keras.models import load_model

In [3]:
# Load the saved model
model = load_model('my_model.h5')

In [4]:
# Define the target shape for input spectrograms
target_shape = (128, 128)

In [5]:
# Define your class labels
classes = ['ali_ghaderi', 'amin_taheri', 'faezeh_najafi', 'houman', 'kourosh',
            'mahdi', 'mahdi_joozdani', 'mani_hosseini', 'mehdi_gozali', 'mojtaba',
              'nazanin_hasani', 'negar', 'saba', 'sam', 'samyar_miri']

In [ ]:
# Function to preprocess and classify an audio file
def test_audio(file_path, model):

    # Load and preprocess the audio file
    y, sr = librosa.load(file_path, sr=None)
    y_trimmed, _ = librosa.effects.trim(y, top_db=30)

    # Extract mel spectrogram
    S = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128 * 5)
    S_db_mel = librosa.amplitude_to_db(S, ref=np.max)

    # Normalize the spectrogram
    S_db_mel_normalized = (S_db_mel - np.min(S_db_mel)) / (np.max(S_db_mel) - np.min(S_db_mel))

    mel_spectrogram = S_db_mel_normalized
    
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

In [20]:
# Test an audio file
test_audio_file = 'mani.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [21]:
# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

Class: ali_ghaderi, Probability: 0.0007
Class: amin_taheri, Probability: 0.0000
Class: faezeh_najafi, Probability: 0.0002
Class: houman, Probability: 0.0000
Class: kourosh, Probability: 0.0000
Class: mahdi, Probability: 0.0003
Class: mahdi_joozdani, Probability: 0.0000
Class: mani_hosseini, Probability: 0.9949
Class: mehdi_gozali, Probability: 0.0001
Class: mojtaba, Probability: 0.0000
Class: nazanin_hasani, Probability: 0.0000
Class: negar, Probability: 0.0037
Class: saba, Probability: 0.0000
Class: sam, Probability: 0.0000
Class: samyar_miri, Probability: 0.0000


In [22]:
# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

The audio is classified as: mani_hosseini
Accuracy: 0.9949
